In [2]:
from dash import Dash, html, dcc, Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd

In [ ]:
Medical_Image_Data = pd.read_csv('data/Medical_Image_Data_01.csv', encoding='CP949')
Patient_Diagnosis_Data = pd.read_csv('data/Patient_Diagnosis_Data.csv')
Patient_Surgery_Data = pd.read_csv('data/Patient_Surgery_Data.csv')

In [ ]:
t1 = Medical_Image_Data.merge(Patient_Diagnosis_Data, on = '환자ID')
t2 = t1.merge(Patient_Surgery_Data, on = '환자ID')
df = t2.drop(columns = ['신장_y', '연령_y', '입원일자_y', '체중_y', '퇴원일자_y', '헤모글로빈수치_y', '환자통증정도_y', '통증기간(월)_y'])

df['통증기간(월)_x'].fillna(df['통증기간(월)_x'].median(), inplace = True)
df['수술시간'].fillna(df['수술시간'].median(), inplace = True)

df['지방축적도'].fillna(df['지방축적도'].median(), inplace = True)
df['PI'].fillna(df['PI'].median(), inplace = True)
df['PT'].fillna(df['PT'].median(), inplace = True)
df['Seg Angle(raw)'].fillna(df['Seg Angle(raw)'].median(), inplace = True)
df['디스크단면적'].fillna(df['디스크단면적'].median(), inplace = True)
df['헤모글로빈수치_x'].fillna(df['헤모글로빈수치_x'].median(), inplace = True)

df['직업'].fillna('기타', inplace = True)
df['수술기법'].fillna('기타', inplace = True)
df['가족력'].fillna(0, inplace = True)

# df['ODI'].fillna(df['ODI'].median(), inplace = True)
df['골밀도'].fillna(df['골밀도'].median(), inplace = True)

df.rename(columns = {'체중_x' : '체중', '신장_x' : '신장', '연령_x' : '연령', '입원일자_x' : '입원일자', '퇴원일자_x' : '퇴원일자',
                     '헤모글로빈수치_x' : '헤모글로빈수치', '환자통증정도_x' : '환자통증정도', '통증기간(월)_x' : '통증기간(월)'}, inplace = True)

df.columns

In [ ]:
def Ygen(x) :
  if x >= 80 :
    return('80~')
  elif x >= 70 :
   return('70~80')
  elif x >= 60 :
   return('60~70')
  elif x >= 50 :
   return('50~60')
  elif x >= 40 :
   return('40~50')
  elif x >= 30 :
    return('30~40')
  elif x >= 20 :
    return('20~30')
  elif x >= 10 :
    return('10~20')
  else :
   return('기타')

df['YY'] = df['연령'].agg(Ygen)
df['YY'].sample(5)

In [ ]:
rr = df.groupby('YY')['재발여부'].sum() / df.groupby('YY')['환자ID'].count() * 100
fig = px.bar(rr, color_discrete_sequence=['#ed7d31'], text_auto='.2s')
fig.update_traces(textposition='outside')
fig.update_layout(paper_bgcolor='rgb(230,231,232)', plot_bgcolor='rgb(230,231,232)', showlegend=False, xaxis=dict(tickangle=0),
                  font={'size':25}, width=900, height=600)
fig.update_xaxes(title_text=None)
fig.update_yaxes(title_text=None, showgrid=False)

In [ ]:
app = JupyterDash(__name__)

In [ ]:
app.layout = html.Div([
    html.H1('Age-Recurrence rate Plot'),
    html.Div([dcc.Graph(figure=fig)])
])

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)